# Overview
This notebook provides the ability to generate random droplet parameters, write them to disk, and 
train a neural network with said droplet parameters to approximate the underlying ODEs that govern 
the droplet parameters.  Once trained researchers can generate a Fortran 90 module that provides 
the ability to estimate droplet radius and temperature for some time in the future.  

The intent is that a small, reasonably trained neural network can provide accurate enough droplet 
characteristic estimations that are significantly faster than an iterative Gauss-Newton technique.
Initial testing indicates a small 4-layer network (roughly 2400 parameters) with Fortran 90 module
generated by this notebook is 30-90x faster than the existing (as of 2024/09/25) iterative approach 
which results in roughly a factor of 2x overall simulation speedup.

This notebook is broken down into the following sections:

1. ODEs of interest
2. Mapping data to/from $[-1, 1]$
3. Generating random droplets
4. Training a neural network
5. Analyzing a network's performance
6. Exporting a network to Fortran 90


# Setup
Since training a neural network to approximate ODEs has multiple workflows (e.g. training a model
vs loading previous models to analyze their performance) there are several variables that need to
be set to exercise all of this notebook's functionality.  In particular, the following should
be reviewed and set depending on which workflows are of interest:

- Loading a previously trained model: `load_model_flag`, `model_load_path`
- Training a new model: `train_model_flag`, `training_data_path`, `number_epochs`
- Saving a newly trained model: `save_model_path`

These should be set at the top of the notebook.  Please adhere to the modification instructions
where the variables are defined.

## Training Data
Training data is not included in the repository due to size, though creating data on the fly
using `create_training_file()` is slow since it is single threaded and this notebook does
not implement a parallel data generation process.

Those who have access to Dr. Richter's group network drive can download previously created 
data sets.  Users external to the group can either slowly create data using the tools in this 
notebook or use scripts exported from an earlier version of this notebook 
(`generate_training_data.py` and `loop-training-data.sh`) to generate data in bulk on
multiple cores.

## Python Dependencies
The following Python packages are needed to exercise full functionality in this notebook 
(versions tested in parentheses):

- Python3 (3.11.5)
- Matplotlib (3.7.1)
- NumPy (1.24.3)
- Pandas (1.5.3)
- PyTorch (2.3.0)

There is no fundamental dependence on any particular version of the dependencies and, barring
any bugs encountered with the packages' main APIs, newer versions are expected to work.

# Notebook Care and Feeding
Notebooks are great for exploring ideas and rapidly iterating to a solution.  They are less
great for maintaining a "production" workflow that needs to be used by multiple people
on a semi-regular basis.  As such, the following guidelines should be followed when making
updates to this so as to preserve everyone's sanity:

- Do not commit the notebook with outputs from a previous run.  This greatly reduces the
  file size in the repository and avoids unnecessary changes when someone re-runs a cell
  whose output changes on each execution.
- The notebook should always allow execution of all cells without errors.  Restarting the
  kernel and running all cells should be run to confirm this (then restart and clear output).

# Future Work

## Hyperparameter Search on the Neural Network's Architecture
The network architecture and hyperparameters were chosen because they:

1. Resulted in a small network
2. Had sizes that *should be* efficient to work with on the CPU

A very limited hyperparameter search has settled on parameters that appear to reliably train
performant networks though the path from start to where we're at was very ad hoc.  Fiddling
with hyperparameters stopped as soon as a network that was reasonably accurate (and was
fairly reproducible) and fast enough when implemented in Fortran. 

A more thorough exploration of the following could be done in an attempt to generate a more accurate
neural network, or a smaller (faster) network so more particles could be simulated without impacting
simulation run-times.

Areas to explore include (roughly in priority order):

1. Number and size of MLP-layers
2. Learning rate and schedule
3. Weights regularization (e.g. L1 or L2 penalties)

## Improving Neural Network Performance
There are a handful of non-architecture/hyperparameter-related things to explore to improve
performance.  Unless otherwise specified, these are all speculations on Greg's part and aren't
a sure bet.

### Train on Sequences of Integration Times for the Same Parameter
Currently training data 

### Normalize the Neural Networks's Integration Time Parameter
Currently all of the neural network's inputs, except for integration time, are normalized into
the range $[-1, 1]$.  Integration time remains in the range of $(0, 10)$ with a focus on $[10^{-3}, 10]$
so as to cover both DNS and LES simulations.  The decision to leave integration time unnormalized
was by accident rather than a conscious choice.

That said, it is unknown as to whether this negatively impacts the performance of the model.  Since
DNS simulations focus on smaller time steps (in the range of $[10^{-3}, 10^{-1})$) and LES simulations on 
larger (in the range of $(10^{-1}, 10)$) it may be worthwhile to perform the same log-scale normalization
to the integration time as is done for the droplet's radius and salinity.  The thought is that the
network would be less sensitive to the scale separation between DNS and LES time steps and learn
them equally.  That said, no quantitative analysis has been performed that would suggest this - it is
purely an unfounded hypothesis at this point.

## Factor This Notebook into a Python Module
This notebook was developed in pieces as the concept of using a neural network was explored and
evaluated for viability.  At the time of initial development (September 2024) having a single
model that is accurate enough is sufficient to update NTLP and move on to other things, so factoring
this notebook was not a priority during the implementation effort.  Given that the models trained
are approximating a fixed set of ODEs, the data distribution they need to learn is essentially
forever fixed which greatly reduces the need to factor this out to enable more streamlined workflows.

Said differently, since training a new model will be rather rare, this notebook is an acceptable
vehicle for repeating the current process with slight tweaks (e.g. different ODEs, modified 
parameter normalization techniques, different parameter counts).  Factoring a Python module would
make it easy to automate certain activities (e.g. data generation and large-scale quantitative model
evaluation) but then requires managing the installation process so it's available to both notebooks
and other scripts.

In [ ]:
import warnings

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.integrate import solve_ivp
import torch
import torch.nn as nn

In [ ]:
#
# NOTE: Do not change this cell directly!  Add a cell below and set new values for the
#       variables of interest.  This makes it significantly easier to revert your changes
#       when commiting changes to the repository - just delete the next cell.
#

# Change this to False if you've previously trained a model and want to to evaluate
# its performance rather than a pre-trained model from disk
load_model_flag = True

# Path to the model, relative to this notebook, to load for performance analysis.
model_load_path = "../models/mlp_4layer-b=1024-lr=1e-3_halvingschedule-l2reg=1e-6.pth"

# Change this to True if you want to train a model from scratch (loading and continuing
# training is not supported yet).  Since generating sufficient training data is typically
# longer than most researchers are willing to wait (O(1 hour) minimum with multiple cores)
# the training_data_path variable must also be set.
train_model_flag = False

# Path to previously created droplet parameters.
#
# NOTE: You must update this to where you've created/copied the training data!
#
# training_data_path = "../data/time_log_spaced.data"
training_data_path = None

# Number of times the model should see the entirety of the training data
# before stopping the optimization process.
number_epochs      = 10

# Name of the currently trained model.  This is used when writing the model's weights
# as a Fortran module.  Default to the name of the loaded model, sans extension, as
# that should match the default training configuration that generated it.
model_name = model_load_path.split( "/" )[-1].split( "." )[0]

# Path to the model, relative to this notebook, to save newly trained models.
#
# NOTE: We disable saving by default to avoid accidentally overwriting an existing model!
#
#model_save_path = "../models/mlp_4layer-b=1024-lr=1e-3_halvingschedule-l2reg=1e-6.pth"
model_save_path = None

# Change this if the droplet_model.f90 module should be created.  This will be
# generated from either 1) a newly trained model or 2) a previously trained, loaded
# model, in that order.
write_weights_flag = False

In [ ]:
# Example training configuration.  Uncomment the variable assignments below to use.
# This trains a file from data in the ../data/ directory and saves the weights in
# the ../models/ directory.  It does not write out a new droplet_model.f90 module.

#train_model_flag   = True
#training_data_path = "../data/time_log_spaced-combined.data"
#model_save_path    = "../models/NEW-mlp_4layer-b=1024-lr=1e-3_halvingschedule-l2reg=1e-6.pth"
#model_name         = model_save_path.split( "/" )[-1].split( "." )[0]
#model_load_path    = model_save_path

In [ ]:
# Force NumPy to print more values on each line rather than wrapping
# around 80 characters.
np.set_printoptions( linewidth=120 )

# Differential Equations of Interest
Example code that solves for a droplet's radius and temperature given its current radius and temperature
as well as the environment it is in, described by:

- The droplet's salinity
- Air temperature
- Relative humidity
- $\rho_a$

In [ ]:
def dydt( t, y, parameters ):
    """
    Differential equations governing a water droplet's radius and temperature as
    a function of time, given the specified physical parameters.
    
    Adapted from code provided by David Richter (droplet_integrator.py) in August 2024.
    Changes made include:
    
      - Accepting NumPy arrays for the parameters instead of having hard-coded values.
      
        NOTE: While it allows for multiple parameters to be specified via additional rows
              this does not necessarily work with scipy.solve_ivp()!
              
      - Parameters are promoted to 64-bit floating point values so all internal
        calculations are performed at maximum precision, resulting in 64-bit outputs.
        It is the caller's responsibility for casting the results to a different precision.
    
    Takes 3 arguments:
    
      t          - Unused argument.  Required for the use of scipy.solve_ivp().
      y          - NumPy array of droplet radii and temperatures.  May be specified
                   as either a 1D vector (of length 2), or a 2D array (sized
                   number_droplets x 2) though must be shape compatible with the
                   parameters array.
      parameters - NumPy array of droplet parameters containing salinity, air temperature,
                   relative humidity, and rhoa.  May be specified as either a
                   1D vector (of length 2), or a 2D array (sized number_droplets x 2)
                   though must be shape compatible with the y array.
    
    Returns 2 values:
    
      dradius_dt      - The derivative of the droplet's radius with respect to time, shaped
                        number_droplets x 1.
      dtemperature_dt - The derivative of the droplet's temperature with respect to time,
                        shaped number_droplets x 1.
    
    """
    
    #
    # NOTE: We work in 64-bit precision regardless of the input
    #       so we get an as accurate as possible answer.
    #
    
    m_s  = parameters[..., 0].astype( "float64" )
    Tf   = parameters[..., 1].astype( "float64" )
    RH   = parameters[..., 2].astype( "float64" )
    rhoa = parameters[..., 3].astype( "float64" )
    
    rhow = np.float64( 1000 )
    Cpp  = np.float64( 4190 )
    Mw   = np.float64( 0.018015 )
    Ru   = np.float64( 8.3144 )
    Ms   = np.float64( 0.05844 )
    Gam  = np.float64( 7.28e-2 )
    Ion  = np.float64( 2.0 )
    Os   = np.float64( 1.093 )
    Shp  = np.float64( 2 )
    Sc   = np.float64( 0.61 )
    Pra  = np.float64( 0.71 )
    Cpa  = np.float64( 1006.0 )
    nuf  = np.float64( 1.57e-5 )
    Lv   = np.float64( (25.0 - 0.02274*26)*10**5 )
    Nup  = np.float64( 2 )

    einf = 611.2*np.exp(17.67*(Tf-273.15)/(Tf-29.65))

    qinf = 0.75/rhoa*(einf*Mw/Ru/Tf)


    qinf = RH/rhoa*(einf*Mw/Ru/Tf);

    Volp = 4/3*np.pi*y[0]**3
    rhop = (m_s + Volp*rhow)/Volp
    taup = rhop*(2*y[0])**2/18/nuf/rhoa


    qstar = einf*Mw/Ru/y[1]/rhoa*np.exp(Lv*Mw/Ru*(1/Tf - 1/y[1]) + 2*Mw*Gam/Ru/rhow/y[0]/y[1] - Ion*Os*m_s*(Mw/Ms)/(Volp*rhow))

    dy1dt = 1/9*Shp/Sc*rhop/rhow*y[0]/taup*(qinf - qstar)
    dy2dt = -1/3*Nup/Pra*Cpa/Cpp*rhop/rhow/taup*(y[1] - Tf) + 3*Lv/y[0]/Cpp*dy1dt

    return [dy1dt, dy2dt]

def solve_ivp_float32_outputs( dydt, t_span, y0, **kwargs ):
    """
    Solves an initial value problem and returns the solutions in 32-bit precision.
    
    NOTE: This is a wrapper around scipy.integrate.solve_ivp(), so it takes the
          same arguments and returns the same values.  See that function's
          help for a (way more) detailed explanation of each argument and value.
    
    Takes 4 arguments:
    
      dydt   - Right-hand side of the system to solve.  The calling signature
               is 'dydt( t, y, parameters )'.
      t_span - 2-member sequence specifying the interval of integration.  dydt
               is integrated from t_span[0] until t_span[1].
      y0     - Initial state of the system to solve.  Must be compatible with
               dydt.
      kwargs - Optional keyword arguments to supply to solve_ivp().
    
    Returns 1 value:
    
      solution - Object containing fields related to the integration process,
                 including:
                 
                   t - Vector of times, of length number_times, where dydt was
                       evaluated.
                   y - Array of solutions, shaped number_variables x numbe_times,
                       where dydt was evaluated.
                       
                 Solutions will be returned according to the evaluation window,
                 t_eval, either supplied via kwargs or selected as a default from
                 t_span.
      
    """

    # Solve the ODE in the precision supplied by the caller.
    solution = solve_ivp( dydt, t_span, y0, **kwargs )

    # Return the outputs as the requested precision.
    solution.t = solution.t.astype( "float32" )
    solution.y = solution.y.astype( "float32" )

    return solution

def plot_droplet_size_temperature( size_temperatures, times ):
    """
    Plots a droplet's size and temperature as a function of time.  
    
    Takes 2 arguments:
    
      size_temperatures - NumPy array, sized 2 x number_times, containing
                          droplet radii and temperatures in the first and second
                          columns, respectively.
      times             - NumPy vector, of length number_times, containing the
                          times corresponding to each entry in size_temperatures.
      
    Returns 2 values:
    
      fig_h - Figure handle created.
      ax_h  - Sequence of two axes handles, one for the size plot and another
              for the temperature plot.

    """

    fig_h, ax_h = plt.subplots( 1, 2, figsize=(9, 4) )

    fig_h.suptitle( "Droplet Size and Temperature (Truth)" )

    ax_h[0].plot( times, size_temperatures[0, :], label="radius" )
    ax_h[0].set_xlabel( "Time (s)" )
    ax_h[0].set_ylabel( "Radius (m)" )
    ax_h[0].set_xscale( "log" )
    ax_h[0].set_yscale( "log" )
    
    ax_h[1].plot( times, size_temperatures[1, :], label="temp" )
    ax_h[1].set_xlabel( "Time (s)" )
    ax_h[1].set_ylabel( "Temperature (K)" )
    ax_h[1].set_xscale( "log" )
    
    return fig_h, ax_h

Solve the ODEs for a set of parameters in the middle of each of the valid ranges.

In [ ]:
# Initial values for the droplet.
#
# NOTE: We must use a list for the initial values rather than a NumPy array
#       as an array will somehow cause a divide by zero.  I did not have
#       enough time to figure out why this is the case.
#
y0 = [0, 0]
y0[0] = np.float32( 1e-4 )  # Radius in logspace(-8,-3)
y0[1] = np.float32( 293 )   # Temperature in (273,310)

# Environmental parameters to solve the ODEs with.
#
# Salinity (m_s)         ~ logspace(-22, -10)
# Air temperature (Tf)   ~ linspace(273, 310)
# Relative humidity (RH) ~ linspace(0.65, 1.1)
# rhoa                   ~ linspace(0.8, 1.2)
parameters = np.array( [1e-18, 291.0, 0.7, 1.0] )

# The ODEs are valid over t_span and solutions are returned over t_eval.
t_span = (0, 10)
t_eval = np.linspace( 0, 10, 1000 )

#
# NOTE: vectorized=True doesn't allow calling with arrays, but it lets the ODE solver
#       operate on multiple values at once if it chooses to.
#
solution = solve_ivp_float32_outputs( dydt, t_span, y0, method="Radau", t_eval=t_eval, args=(parameters,) )

plot_droplet_size_temperature( solution.y, solution.t )

# Mapping Droplet Parameters to/from $[-1, 1]$
Create routines for moving between the normal range of physical parameters and
a mapping on the range [-1, 1].  The ODEs are solved with physical parameters,
each with their own dynamic range, while the model operates on parameters that
each have the same range.

In [ ]:
# Provide the range of each of the input parameters so we can normalize
# each to [-1, 1], allowing the models to weight each equally.  If we don't
# perform this mapping then optimization process will effectively ignore
# the small parameters (e.g. radius) as they won't contribute as much as
# the big parameters (e.g. temperature).  As a result, we use log-scale
# for the parameters that have a large dynamic range.
DROPLET_RADIUS_LOG_RANGE          = np.array( (-8, -3) )
DROPLET_TEMPERATURE_RANGE         = np.array( (273, 310) )
DROPLET_SALINITY_LOG_RANGE        = np.array( (-22, -10) )
DROPLET_AIR_TEMPERATURE_RANGE     = np.array( (273, 310) )
DROPLET_RELATIVE_HUMIDITY_RANGE   = np.array( (0.65, 1.1) )
DROPLET_RHOA_RANGE                = np.array( (0.8, 1.2) )

def normalize_droplet_parameters( droplet_parameters ):
    """
    Normalizes an array of droplet parameters into the range [-1, 1].  Operates
    on both input and parameters.
    
    Takes 1 argument:

      droplet_parameters - NumPy array of parameters to normalize, shaped 
                           number_droplets x number_parameters.  May either be
                           1D, for a single droplet, or 2D for multiple droplets.
                           number_parameters must be either 2 (output parameters),
                           6 (input parameters without t_final), or 7 (input
                           parameters with t_final).

    Returns 1 value:
    
      normalized_droplet_parameters - NumPy array of normalized parameters, shaped
                                      number_droplets x number_parameters.  Has the
                                      same shape as droplet_parameters.

    """
    
    #
    # NOTE: We take care to handle arbitrary rank arrays!  While we expect either 
    #       rank-1 or rank-2, the code is written to handle larger ranks naturally
    #       with the inner dimension representing a single droplet's parameters.
    #
    
    number_parameters = droplet_parameters.shape[-1]

    # Bail if we didn't get 1) output parameters, 2) input parameters without a t_final,
    # or 3) input parameters with a t_final.
    if number_parameters != 2 and number_parameters != 6 and number_parameters != 7:
        raise ValueError( "Unknown number of parameters to normalize ({:d})!".format( number_parameters ) )

    normalized_droplet_parameters = np.empty_like( droplet_parameters )
    
    # We always have radius and temperature.
    normalized_droplet_parameters[..., 0] = (np.log10( droplet_parameters[..., 0] ) - np.mean( DROPLET_RADIUS_LOG_RANGE )) / (np.diff( DROPLET_RADIUS_LOG_RANGE ) / 2)
    normalized_droplet_parameters[..., 1] = (droplet_parameters[..., 1] - np.mean( DROPLET_TEMPERATURE_RANGE )) / (np.diff( DROPLET_TEMPERATURE_RANGE ) / 2)

    # Sometimes we have the remaining parameters.
    if number_parameters > 2:
        normalized_droplet_parameters[..., 2] = (np.log10( droplet_parameters[..., 2] ) - np.mean( DROPLET_SALINITY_LOG_RANGE )) / (np.diff( DROPLET_SALINITY_LOG_RANGE ) / 2)
        normalized_droplet_parameters[..., 3] = (droplet_parameters[..., 3] - np.mean( DROPLET_AIR_TEMPERATURE_RANGE )) / (np.diff( DROPLET_AIR_TEMPERATURE_RANGE ) / 2)
        normalized_droplet_parameters[..., 4] = (droplet_parameters[..., 4] - np.mean( DROPLET_RELATIVE_HUMIDITY_RANGE )) / (np.diff( DROPLET_RELATIVE_HUMIDITY_RANGE ) / 2)
        normalized_droplet_parameters[..., 5] = (droplet_parameters[..., 5] - np.mean( DROPLET_RHOA_RANGE )) / (np.diff( DROPLET_RHOA_RANGE ) / 2)

        # Copy the evaluation times when they're present.
        if number_parameters > 6:
            normalized_droplet_parameters[..., 6] = droplet_parameters[..., 6]

    return normalized_droplet_parameters

def scale_droplet_parameters( droplet_parameters ):
    """
    Scales an array of droplet parameters from the range [-1, 1] to their
    expected ranges of physical values.  Operates on both input and parameters.
    
    Takes 1 argument:

      droplet_parameters - NumPy array of parameters to scale, shaped 
                           number_droplets x number_parameters.  May either be
                           1D, for a single droplet, or 2D for multiple droplets.
                           number_parameters must be either 2 (output parameters),
                           6 (input parameters without t_final), or 7 (input
                           parameters with t_final).

    Returns 1 value:
    
      scaled_droplet_parameters - NumPy array of scaled parameters, shaped
                                  number_droplets x number_parameters.  Has the
                                  same shape as droplet_parameters.


    """

    #
    # NOTE: We take care to handle arbitrary rank arrays!  While we expect either 
    #       rank-1 or rank-2, the code is written to handle larger ranks naturally
    #       with the inner dimension representing a single droplet's parameters.
    #

    number_parameters = droplet_parameters.shape[-1]

    # Bail if we didn't get 1) output parameters, 2) input parameters without a t_final,
    # or 3) input parameters with a t_final.
    if number_parameters != 2 and number_parameters != 6 and number_parameters != 7:
        raise ValueError( "Unknown number of parameters to scale ({:d})!".format( number_parameters ) )

    scaled_droplet_parameters = np.empty_like( droplet_parameters )
    
    # We always have radius and temperature.
    scaled_droplet_parameters[..., 0] = 10.0 ** (droplet_parameters[..., 0] * (np.diff( DROPLET_RADIUS_LOG_RANGE ) / 2) + np.mean( DROPLET_RADIUS_LOG_RANGE ))
    scaled_droplet_parameters[..., 1] = droplet_parameters[..., 1] * (np.diff( DROPLET_TEMPERATURE_RANGE ) / 2) + np.mean( DROPLET_TEMPERATURE_RANGE ) 

    # Sometimes we have the remaining parameters.
    if number_parameters > 2:
        scaled_droplet_parameters[..., 2] = 10.0 ** (droplet_parameters[..., 2] * (np.diff( DROPLET_SALINITY_LOG_RANGE ) / 2) + np.mean( DROPLET_SALINITY_LOG_RANGE ))
        scaled_droplet_parameters[..., 3] = droplet_parameters[..., 3] * (np.diff( DROPLET_AIR_TEMPERATURE_RANGE ) / 2) + np.mean( DROPLET_AIR_TEMPERATURE_RANGE )
        scaled_droplet_parameters[..., 4] = droplet_parameters[..., 4] * (np.diff( DROPLET_RELATIVE_HUMIDITY_RANGE ) / 2) + np.mean( DROPLET_RELATIVE_HUMIDITY_RANGE ) 
        scaled_droplet_parameters[..., 5] = droplet_parameters[..., 5] * (np.diff( DROPLET_RHOA_RANGE ) / 2) + np.mean( DROPLET_RHOA_RANGE )

        # Copy the evaluation times when they're present.
        if number_parameters > 6:
            scaled_droplet_parameters[..., 6] = droplet_parameters[..., 6]
        
    return scaled_droplet_parameters

# Test normalizing output parameters that are...
#
#   1. at the lower edge of the valid input space
#   2. in the middle of the valid input space
#   3. at the upper edge of the valid input space
#   4. below the lower edge of the valid input space
#   5. above the upper edge of the valid input space
#
test_parameters = np.array( [[10**-8, 273],
                             [10**-5.5, 291.5],
                             [10**-3, 310],
                             [10**-9, 250],
                             [10**-2, 320]] )

if not np.allclose( scale_droplet_parameters( normalize_droplet_parameters( test_parameters ) ),
                    test_parameters ):
    raise ValueError( "Scaling normalized outputs was not an inverse operation!" )

# Test normalizing input parameters without a t_final.  The parameters chosen check
# the same cases as the output tests above.
test_parameters_full = np.array( [[10**-8,   273,   10**-22, 273,   0.65,  0.8],
                                  [10**-5.5, 291.5, 10**-16, 291.5, 0.875, 1.0],
                                  [10**-3,   310,   10**-10, 310,   1.1,   1.2],
                                  [10**-9,   250,   10**-23, 250,   0.5,   0.7],
                                  [10**-2,   320,    10**-9, 320,   1.2,   1.3]] )

if not np.allclose( scale_droplet_parameters( normalize_droplet_parameters( test_parameters_full ) ),
                    test_parameters_full ):
    raise ValueError( "Scaling normalized inputs (without t_final) was not an inverse operation!" )
    
# Test normalizing input parameters with a t_final.  The parameters chosen check
# the same cases as the output tests above.
test_parameters_full = np.array( [[10**-8,   273,   10**-22, 273,   0.65,  0.8, 10**-3],
                                  [10**-5.5, 291.5, 10**-16, 291.5, 0.875, 1.0, 10**-2],
                                  [10**-3,   310,   10**-10, 310,   1.1,   1.2, 10**1],
                                  [10**-9,   250,   10**-23, 250,   0.5,   0.7, 10**-3.2],
                                  [10**-2,   320,    10**-9, 320,   1.2,   1.3, 10**1.1]] )

if not np.allclose( scale_droplet_parameters( normalize_droplet_parameters( test_parameters_full ) ),
                    test_parameters_full ):
    raise ValueError( "Scaling normalized inputs (with t_final) was not an inverse operation!" )

# GeneratingTraining Data
We want to create an on-disk training data set to streamline the training process.
Not training a model itself, per se, but rather making it easy to quickly
explore different models that are comparable without having questions about
what data were seen by each model.

A secondary benefit to this is that the training process is much faster as
we don't have to worry about the ability to quickly generate random data (which
requires solving ODEs) and slowing down the actual training loop.

In [ ]:
# Demonstrate that we can generate input parameters, without t_final, by sampling [-1, 1]
# and scaling them to their physical range.
#
# NOTE: This does not generate random t_sample as the original code was not developed with
#       that in mind.  Partially because the distribution is different and partially because
#       t_final is not currently normalized on input to the model.  This should be evaluated.
#
random_inputs = scale_droplet_parameters( np.reshape( np.random.uniform( -1, 1, 30 ), (5, 6) ) )

print( random_inputs )

In [ ]:
def create_droplet_batch( number_droplets, linear_time_flag=False, number_evaluations=1 ):
    """
    Creates a batch of random droplets' input parameters, with t_final.  t_final is sampled
    from a slightly larger distribution than the anticipated use cases (spanning [1e-3, 1e1])
    so as to increase the performance at the edges of the range.
    
    Weird parameters encountered, both as inputs to the ODEs and the outputs returned, are
    logged and replaced with valid inputs and outputs to ensure the batch is suitable
    for training.  Logging returns lists of parameters categorized by the reason they were
    filtered out.
    
    The following categories exist for weird input parameters:
    
      evaluation_warning - The parameters generated a floating point exception during
                           evaluation of the ODEs
      failed_solve       - The ODEs failed to converge as a small enough timestep could
                           not be identified
      invalid_inputs     - XXX: Greg can't remember what triggered this and scipy.solve_ivp()
                                can raise ValueError in a number of ways

    The following categories exist for weird output parameters:
    
      radius_nonpositive      - The ODEs generated a physically impossible, negative radius
      radius_too_small        - The ODEs generated a radius smaller than the expected range
      radius_too_large        - The ODEs generated a radius larger than the expected range
      temperature_nonpositive - The ODEs generated a physically impossible, negative temperature
      temperature_too_small   - The ODEs generated a temperature smaller than the expected range
      temperature_too_large   - The ODEs generated a temperature larger than the expected range

    NOTE: The use of linear_time_flag==True leads to a poorly sampled t_final parameter
          that results in poor model performance for DNS time scales (i.e. t_final in 
          [1e-3, 1e-1]) unless trained on a *lot* of droplets.  The default leads to
          decent performance on both DNS time scales and LES time scales (i.e. t_final
          in [1e-1, 1e1]).

    Takes 3 arguments:

      number_droplets    - Number of droplets to generate parameters for.
      linear_time_flag   - Optional boolean specifying whether integration times should
                           be sampled uniformly through the time range or log spaced.
                           Defaults to False so that short term dynamics are captured.
      number_evaluations - Optional number of integration times to evaluate each
                           parameter at so that a temporal window can be learned.  If
                           omitted defaults to a single point in time per parameter.

    Returns 5 values:

      random_inputs     - Array, sized number_droplets x 6, containing the droplets
                          radii, temperatures, salinity, air temperature, relative
                          humidity, and rhoa.
      random_outputs    - Array, sized number_droplets x 2, containing the droplets
                          radii and temperatures.
      integration_times - Array, sized number_droplets, containing the times corresponding
                          to the associated random_inputs and random_outputs.
      weird_inputs      - Dictionary containing lists of weird input parameters.  Each
                          key represents a category of "weird".
      weird_outputs     - Dictionary containing lists of weird output parameters.  Each
                          key represents a category of "weird".
      
    """

    # Promote run-time warnings to errors so we get an exception whenever the ODEs
    # are evaluated in problematic corners of the parameter space.  This not only
    # prevents them from showing up on standard error but also lets us log them and
    # ignore them so we only sample the "good" parts of the space.
    warnings.simplefilter( "error", RuntimeWarning )

    random_inputs     = np.empty( (number_droplets * number_evaluations, 6), dtype=np.float32 )
    random_inputs[::number_evaluations, :] = scale_droplet_parameters( np.reshape( np.random.uniform( -1, 1, number_droplets*6 ),
                                                                      (number_droplets, 6) ).astype( "float32" ) )
    # Duplicate each unique droplet parameter once for each evaluation.
    # This keeps them in parameter order.
    for droplet_index in np.arange( number_droplets ):
        start_index = droplet_index * number_evaluations
        end_index   = (droplet_index + 1) * number_evaluations

        random_inputs[start_index+1:end_index, :] = random_inputs[start_index, :]

    random_outputs = np.empty_like( random_inputs, shape=(number_droplets * number_evaluations, 2) )
    
    # Size of the time window to sample when we're generating multiple temporal
    # evaluations.  In linear units when linear_time_flag==True (e.g. 1 second),
    # otherwise in log-space (e.g. 1 order of magnitude).
    TIME_WINDOW_SIZE = 1

    # We generate data for a time window that is slightly larger than what we're interested
    # in (logspace( -3, 1 )) so we can learn the endpoints.
    TIME_RANGE = (10.0**-3.2, 10.0**1.1)

    integration_times = np.empty_like( random_inputs, shape=(number_droplets * number_evaluations) )
    if linear_time_flag:
        # Generate the starting point for each of the evaluation groups.  We
        # construct the other times in the group below.
        integration_times[::number_evaluations] = np.random.uniform( TIME_RANGE[0], TIME_RANGE[1], number_droplets ).astype( "float32" )

        if number_evaluations > 1:
            for droplet_index in np.arange( number_droplets ):
                start_index = droplet_index * number_evaluations
                end_index   = (droplet_index + 1) * number_evaluations

                # Generate the remaining points in this group while taking care
                # to not go beyond the largest time we're allowed.
                integration_times[start_index:end_index] = np.linspace( integration_times[start_index],
                                                                        min( integration_times[start_index] + TIME_WINDOW_SIZE, TIME_RANGE[1] ),
                                                                        number_evaluations )
                
    else:
        # Generate the starting point for each of the evaluation groups.  We
        # construct the other times in the group below.
        integration_times[::number_evaluations] = (10.0**np.random.uniform( np.log10( TIME_RANGE[0] ),
                                                                            np.log10( TIME_RANGE[1] ),
                                                                            number_droplets )).astype( "float32" )
                                                                            
        if number_evaluations > 1:
            for droplet_index in np.arange( number_droplets ):
                start_index = droplet_index * number_evaluations
                end_index   = (droplet_index + 1) * number_evaluations

                # Generate the remaining points in this group while taking care
                # to not go beyond the largest time we're allowed.
                starting_exponent = np.log10( integration_times[start_index] )
                integration_times[start_index:end_index] = 10.0**np.linspace( starting_exponent,
                                                                              min( starting_exponent + TIME_WINDOW_SIZE, np.log10( TIME_RANGE[1] ) ),
                                                                              number_evaluations )

    # We count the number of problematic parameters we encounter.  Additionally,
    # track the reason why we found the parameters problematic along with the
    # parameters themselves.
    number_warnings = 0
    weird_inputs    = { "evaluation_warning": [],
                        "failed_solve":       [],
                        "invalid_inputs":     [] }
    weird_outputs   = { "radius_nonpositive":      [],
                        "radius_too_small":        [],
                        "radius_too_large":        [],
                        "temperature_nonpositive": [],
                        "temperature_too_small":   [],
                        "temperature_too_large":   [] }

    # XXX: Need to figure out if we can simply evaluate the ODEs at number_evaluations-many points
    #      and re-roll only the points that are outside the expected ranges.  Right now we evaluate
    #      the ODEs for every t_final even when we could evaluate it once for a list of multiple so
    #      that we discard all of the evaluations when one of them is problematic.
    for droplet_index in np.arange( number_droplets * number_evaluations ):
        
        # Emulate a do/while loop so we always evaluate at least one parameter before
        # deciding whether to keep it or not.
        while True:
            y0         = random_inputs[droplet_index, :2]
            parameters = random_inputs[droplet_index, 2:]
            t_final    = integration_times[droplet_index]
    
            try:
                solution = solve_ivp( dydt, [0, t_final], y0, method="Radau", t_eval=[t_final], args=(parameters,) )

                if solution.success:
                    random_outputs[droplet_index, 0] = solution.y[0][0]
                    random_outputs[droplet_index, 1] = solution.y[1][0]

                    good_parameters_flag = True
                    
                    # Check that we didn't get a physically impossible solution.  These
                    # will be logged and we'll reroll the dice to replace them.
                    #
                    # NOTE: We don't strictly need to check for negative temperatures as
                    #       that will get covered in validate_output_parameters() but
                    #       we leave it here so it is easy to identify physically impossible
                    #       cases.
                    #
                    if solution.y[0][0] <= 0.0:
                        weird_outputs["radius_nonpositive"].append( [y0, parameters, t_final, random_outputs[droplet_index, :]] )
                        good_parameters_flag = False
                    elif solution.y[1][0] <= 0.0:
                        weird_outputs["temperature_nonpositive"].append( [y0, parameters, t_final, random_outputs[droplet_index, :]] )
                        good_parameters_flag = False

                    # Check that we didn't get strange solutions that are outside of the
                    # expected ranges.  These will also be logged and replaced.
                    if solution.y[0][0] < 10.0**DROPLET_RADIUS_LOG_RANGE[0] * (100 - 3) / 100:
                        weird_outputs["radius_too_small"].append( [y0, parameters, t_final, random_outputs[droplet_index, :]] )
                        good_parameters_flag = False
                    elif solution.y[0][0] > 10.0**DROPLET_RADIUS_LOG_RANGE[1] * (100 + 3) / 100:
                        weird_outputs["radius_too_large"].append( [y0, parameters, t_final, random_outputs[droplet_index, :]] )
                        good_parameters_flag = False

                    if solution.y[1][0] < DROPLET_TEMPERATURE_RANGE[0] * (100 - 3) / 100:
                        weird_outputs["temperature_too_small"].append( [y0, parameters, t_final, random_outputs[droplet_index, :]] )
                        good_parameters_flag = False
                    elif solution.y[1][0] > DROPLET_TEMPERATURE_RANGE[1] * (100 + 3) / 100:
                        weird_outputs["temperature_too_large"].append( [y0, parameters, t_final, random_outputs[droplet_index, :]] )
                        good_parameters_flag = False

                    # Jump to the next droplet's parameters if we didn't detect a problem.
                    if good_parameters_flag:
                        break
                else:
                    # Record the cases that fail to converge.
                    weird_inputs["failed_solve"].append( [y0, parameters, t_final, solution.message] )

            except RuntimeWarning as e:
                weird_inputs["evaluation_warning"].append( [y0, parameters, t_final, str( e )] )
            except ValueError as e:
                weird_inputs["invalid_inputs"].append( [y0, parameters, t_final, str( e )] ) 
            
            # We failed to create acceptable parameters.  Reroll the dice for 
            # this droplet and try again.
            random_inputs[droplet_index, :] = scale_droplet_parameters( 
                np.random.uniform( -1, 1, 6 ).astype( "float32" ) )
    
    # XXX: Warn users if there were strange droplets?  No simple way to see if any
    #      of the lists associated with weird_*'s keys are non-empty.
    
    return random_inputs, random_outputs, integration_times, weird_inputs, weird_outputs

def merge_weird_parameters( parameters_1, parameters_2 ):
    """
    Merge two dictionaries of lists into a separate copy containing the
    concatenated lists of both.  All list entries of the first dictionary
    come before the first entry of the second dictionary when the first
    has a non-empty list.
    
    Takes 2 arguments:
    
      parameters_1 - 1st dictionary of lists to merge.
      parameters_2 - 2nd dictionary of lists to merge.
      
    Returns 1 value:
    
      merged_parameers - Dictionary containing the merged lists.

    """

    merged_parameters = parameters_1.copy()

    for type_name, weird_things in parameters_2.items():
        if type_name in merged_parameters:
            merged_parameters[type_name].extend( weird_things )
        else:
            merged_parameters[type_name] = weird_things

    return merged_parameters

def write_weird_parameters_to_spreadsheet( file_name, weird_inputs, weird_outputs ):
    """
    Creates an Excel file containing one spreadsheet per type of weird inputs or
    outputs.
    
    See create_droplet_batch() for details on weird droplet inputs and outputs.

    Takes 3 arguments:

      file_name     - Path to the spreadsheet to create.  If it exists it is
                      overwritten.
      weird_inputs  - Dictionary of lists representing weird input droplet
                      parameters.  
      weird_outputs - Dictionary of lists representing weird output droplet
                      parameters.

    Returns nothing.

    """

    # XXX: move this
    import pandas as pd

    # Open the file for writing, overwriting an existing file if necessary.
    with pd.ExcelWriter( file_name ) as writer:

        # Create a DataFrame for each category of input parameters weirdness.
        for type_name, weird_things in weird_inputs.items():
            sheet_name = "inputs-{:s}".format( type_name )

            # Columns in the DataFrame.
            columns = ["initial_radius",
                       "initial_temperature",
                       "salinity",
                       "air_temperature",
                       "relative_humidity",
                       "rhoa",
                       "t_final",
                       "error"]

            # Convert our list of lists of NumPy arrays to lists of lists
            # so spreadsheet cells contain a single value.
            weird_things_listified = []
            for weird_thing in weird_things:
                weird_thing_listified = []

                weird_thing_listified.extend( weird_thing[0].tolist() )
                weird_thing_listified.extend( weird_thing[1].tolist() )
                weird_thing_listified.extend( weird_thing[2:] )

                weird_things_listified.append( weird_thing_listified )

            # Build a DataFrame and write it as a new sheet.
            filtered_df = pd.DataFrame( weird_things_listified, columns=columns )
            filtered_df.to_excel( writer, sheet_name=sheet_name, index=False )

        # Create a DataFrame for each category of output parameters weirdness.
        for type_name, weird_things in weird_outputs.items():
            sheet_name = "outputs-{:s}".format( type_name )

            # Columns in the DataFrame.
            columns = ["initial_radius",
                       "initial_temperature",
                       "salinity",
                       "air_temperature",
                       "relative_humidity",
                       "rhoa",
                       "t_final",
                       "radius",
                       "temperature"]

            # Convert our list of lists of NumPy arrays to lists of lists
            # so spreadsheet cells contain a single value.
            weird_things_listified = []
            for weird_thing in weird_things:
                weird_thing_listified = []

                weird_thing_listified.extend( weird_thing[0].tolist() )
                weird_thing_listified.extend( weird_thing[1].tolist() )
                weird_thing_listified.append( weird_thing[2] )
                weird_thing_listified.extend( weird_thing[3].tolist() )

                weird_things_listified.append( weird_thing_listified )

            # Build a DataFrame and write it as a new sheet.
            filtered_df = pd.DataFrame( weird_things_listified, columns=columns )
            filtered_df.to_excel( writer, sheet_name=sheet_name, index=False )
            
def create_training_file( file_name, number_droplets, weird_file_name=None, user_batch_size=None ):
    """
    Generates random droplet parameters, both inputs and their corresponding
    ODE outputs, and writes them as fixed-size binary records to a file.  This
    makes for efficient access, both sequential and random, for training and
    analysis.
    
    The output file written is comprised of one or more 36-byte records, one
    per droplet.  Each record holds 9x 32-bit, floating point in host-byte
    order (typically little-endian):

      1. Input radius, in meters
      2. Input temperature, in Kelvin
      3. Input salinity, aka the mass of disolved salt, in kilograms
      4. Input air temperature, in Kelvin
      5. Input relative humidity, as a non-dimensional value with 100% humidity at 1.0
      6. Input rhoa, in non-dimensional units
      7. Input evaluation time, in seconds
      8. Output radius, in meters
      9. Output temperature, in Kelvin

    The file generated can be read via read_training_file(). 
    
    Takes 4 arguments:
    
      file_name       - Path to the file to write the droplet parameters.  This
                        is overwritten if it exists.
      number_droplets - The number of droplets to generate.
      weird_file_name - Optional path to write any weird parameters encountered
                        during batch generation.  If specified an Excel spreadsheet
                        file is written, otherwise weird parameters are silently
                        ignored.
      user_batch_size - Optional batch size specifying the number of parameters
                        to generate at once.  If omitted, defaults to a "small"
                        number of parameters that balances memory footprint,
                        time to generate, and file write size.  This does not
                        need to evenly divide number_droplets.
    
    Returns nothing.

    """
    
    # Balance the time it takes to generate a single batch vs the efficiency of
    # writing it out.  Each droplet's parameters takes 36 bytes (9x 32-bit floats
    # comprised of the 7x input parameters and the 2x output parameters).
    if user_batch_size is not None:
        BATCH_SIZE = user_batch_size
    else:
        BATCH_SIZE = 1024 * 10
    
    # Number of batches to create including the last, partial batch when
    # the batch size does not evenly divide the number of droplets.
    number_batches = (number_droplets + BATCH_SIZE - 1) // BATCH_SIZE
    
    # Dictionaries for tracking weird parameters.
    weird_inputs  = {}
    weird_outputs = {}

    with open( file_name, "wb" ) as output_fp:
        for batch_index in range( number_batches ):

            # Determine how many droplets to create in this batch.
            if batch_index != (number_batches - 1):
                batch_size = BATCH_SIZE
            else:
                batch_size = number_droplets % BATCH_SIZE
            
            # Get the next batch of droplets.
            (inputs,
             outputs,
             times,
             batch_weird_inputs,
             batch_weird_outputs) = create_droplet_batch( batch_size )
    
            # Track the weirdness for post-mortem analysis.
            weird_inputs  = merge_weird_parameters( weird_inputs, batch_weird_inputs )
            weird_outputs = merge_weird_parameters( weird_outputs, batch_weird_outputs )                    

            inputs_outputs = np.hstack( (inputs,
                                         times.reshape( (batch_size, 1) ),
                                         outputs) )

            # Serialize the array 
            inputs_outputs.tofile( output_fp )

    if weird_file_name is not None:
        write_weird_parameters_to_spreadsheet( weird_file_name,
                                               weird_inputs,
                                               weird_outputs )

def read_training_file( file_name ):
    """
    Reads all of the fixed-size binary records from the path specified and returns
    NumPy arrays containing input parameters, output parameters, and integration
    times.
    
    Takes 1 arguments:
    
      file_name - Path to the file to parse.
      
    Returns 3 values:
    
      inputs  - NumPy array, shaped number_droplets x 6, containing the input parameters.
      outputs - NumPy array, shaped number_droplets x 2, containing the output parameters.
      times   - NumPy array, shaped number_droplets x 1, containing the integration times.

    """

    inputs_outputs = np.fromfile( file_name, dtype=np.float32 ).reshape( (-1, 9) )
    inputs         = inputs_outputs[:, :6]
    times          = inputs_outputs[:, 6]
    outputs        = inputs_outputs[:, 7:]

    return inputs, outputs, times

def clean_training_data( file_name ):
    """
    Removes invalid droplet parameters found in the supplied file and overwrites it
    so it only contains valid parameters.  Currently this only filters output
    parameters that are outside of the expected ranges by more than 3% on either end.
    
    NOTE: This function isn't normally needed, but there may be times where invalid
          outputs have been written but the vast majority are good and need to be
          salvaged.
    
    Takes 1 argument:
    
      file_name - Path to the file containing droplet parameters written by
                  create_training_file().  This file is overwritten with the
                  filtered droplet parameters.
                  
    Returns 2 values:
    
      number_parameters     - Number of droplet parameters in the original file.
      number_bad_parameters - Number of droplet parameters that were filtered out.
    
    """

    #
    # NOTE: This isn't terribly efficient and assumes there is enough RAM to
    #       read, make a copy, and write back out.
    #
    input_parameters, output_parameters, times = read_training_file( file_name )

    bad_data_mask = ((output_parameters[:, 0] < 10.0**DROPLET_RADIUS_LOG_RANGE[0] * (100 - 3) / 100) |
                     (output_parameters[:, 0] > 10.0**DROPLET_RADIUS_LOG_RANGE[1] * (100 + 3) / 100) |
                     (output_parameters[:, 1] < DROPLET_TEMPERATURE_RANGE[0] * (100 - 3) / 100) |
                     (output_parameters[:, 1] > DROPLET_TEMPERATURE_RANGE[1] * (100 + 3) / 100))

    input_outputs = np.hstack( (input_parameters,
                                times.reshape( (-1, 1) ),
                                output_parameters) )

    # Overwrite the original file with only the good data.
    input_outputs[~bad_data_mask, :].tofile( file_name )

    # Return the counts of good and bad data so the caller knows what was done.
    return input_parameters.shape[0], bad_data_mask.sum()

In [ ]:
# Demonstrate generating a batch of droplets with a small number of evaluations.
number_droplets    = 128
number_evaluations = 4

[inputs,
 outputs,
 integration_times,
 weird_inputs,
 weird_outputs] = create_droplet_batch( number_droplets, number_evaluations=number_evaluations )

number_weird_inputs  = sum( map( lambda type_name: len( weird_inputs[type_name] ), weird_inputs ) )
number_weird_outputs = sum( map( lambda type_name: len( weird_outputs[type_name] ), weird_outputs ) )

print( "Generated {:d} droplet parameter{:s}.\n".format(
    number_droplets,
    "" if number_droplets == 1 else "s" ) )

if number_weird_inputs > 0:
    print( "{:d} weird input{:s}:\n".format(
        number_weird_inputs,
        "" if number_weird_inputs == 1 else "s" ) )
    
    for type_name, weird_things in weird_inputs.items():
        number_weird_things = len( weird_things )
        if number_weird_things == 0:
            continue
            
        print( "    {:d} {:s}".format(
            number_weird_things,
            type_name ) )
        
    print( "" )

if number_weird_outputs > 0:
    print( "{:d} weird output{:s}:\n".format(
        number_weird_outputs,
        "" if number_weird_outputs == 1 else "s" ) )
    
    for type_name, weird_things in weird_outputs.items():
        number_weird_things = len( weird_things )
        if number_weird_things == 0:
            continue
            
        print( "    {:d} {:s}".format(
            number_weird_things,
            type_name ) )
        
    print( "" )
    
write_weird_parameters_to_spreadsheet( "test.xlsx", weird_inputs, weird_outputs )

In [ ]:
# Create a training file with a handful of droplets, filter out any invalid
# parameters (this should no longer happen), and read them back in to
# see the distribution of t_final.  
training_test_path = "foo.data"
weird_test_path    = "foo.xlsx"
create_training_file( training_test_path, 1024, weird_file_name=weird_test_path )

number_parameters, number_bad_parameters = clean_training_data( training_test_path )
print( "Removed {:d} parameter{:s} from {:d} parameter{:s} ({:.2f}%).".format(
    number_bad_parameters,
    "" if number_bad_parameters == 1 else "s",
    number_parameters,
    "" if number_parameters == 1 else "s",
    number_bad_parameters / number_parameters * 100.0 ) )

input_parameters, output_parameters, times = read_training_file( training_test_path )

fig_h, ax_h = plt.subplots( 1, 1 )
ax_h.plot( np.log10( times ), "." )
ax_h.set_xlabel( "Droplet #" )
ax_h.set_ylabel( "log10( $t_{final}$ )" )
ax_h.set_title( "Distribution of $t_{final}$" )

plt.show()

# Training a Model
We define a simple model architecture in PyTorch, setup our loss function and basic
hyperparameters, and then randomly walk through the training data one or more times.
If configured, the trained model is saved to disk.

In [ ]:
class SimpleNet( nn.Module ):
    """
    4-layer multi-layer perceptron (MLP) with ReLU activations.  This aims to
    balance parameter count vs computational efficiency so that inferencing with 
    it is faster than Gauss-Newton iterative solvers.
    """
    
    def __init__( self) :
        super().__init__()
        
        #
        # NOTE: These sizes were chosen without any consideration other than creating
        #       a small network (wrt parameter count) and should have good computational
        #       efficiency (wrt memory alignment and cache lines).  No effort has been
        #       spent to improve upon the initial guess.
        #
        self.fc1 = nn.Linear( 7, 32 )
        self.fc2 = nn.Linear( 32, 32 )
        self.fc3 = nn.Linear( 32, 32 )
        self.fc4 = nn.Linear( 32, 2 )
        
    def forward( self, x ):
        x = torch.relu( self.fc1( x ) )
        x = torch.relu( self.fc2( x ) )
        x = torch.relu( self.fc3( x ) )
        x = self.fc4( x )
        
        return x

In [ ]:
# Train using a local GPU if we have one, otherwise stay on the CPU.  While
# SimpleNet isn't a large model having GPU acceleration for large batch
# counts and a non-trivial number of droplet parameters (>100 million) certainly
# helps.
device = torch.device( "cuda" if torch.cuda.is_available() else "cpu" )

print( "Training with '{}'.".format( device ) )

# Create an instance of SimpleNet and configure its optimization parameters:
#
#   - We use mean-squared error loss so we chase outliers
#   - We use Adam so we have momentum when performing gradient descent
#   - Given that we have a relatively large batch size we use a large
#     initial learning rate of 1e-3.
#   - We want smaller weights so we use a L2 regularization penalty of 1e-6
#     to encourage that (as well as having non-zero weights rather than
#     leaning heavily on just a subset of weights)
#
model     = SimpleNet()
criterion = nn.MSELoss()
optimizer = torch.optim.Adam( model.parameters(), lr=1e-3, weight_decay=1e-6 )

# Move the model to the device we're training with.
model = model.to( device )

In [ ]:
def train_model( model, criterion, optimizer, device, number_epochs, training_file ):
    """
    Trains the supplied model for one or more epochs using all of the droplet parameters
    in an on-disk training file.  The parameters are read into memory once and then
    randomly sampled each epoch.  Any weird parameters encountered in the training file
    are logged and 
    
    Takes 6 arguments:
    
      model         - PyTorch model to optimize.
      criterion     - PyTorch loss object to use during optimization.
      optimizer     - PyTorch optimizer associated with model.
      device        - Device string indicating where the optimization is 
                      being performed.
      number_epochs - Number of epochs to train model for.  All training
                      data in training_file will be seen by the model 
                      this many times.
      training_file - Path to the file containing training data created by
                      create_training_file().
    
    Returns 1 value:
    
      loss_history - List of training losses, one per mini-batch during
                     the training process.

    """

    #
    # NOTE: This is inefficient and requires the entire training data set to reside in
    #       RAM.  We could read chunks of the file on demand but that would require
    #       a more sophisticated training loop that performs I/O in a separate thread
    #       while the optimization process executes.
    #
    #       TL;DR Find a big machine to train on.
    #
    input_parameters, output_parameters, integration_times = read_training_file( training_file )

    BATCH_SIZE      = 1024
    MINI_BATCH_SIZE = 1024

    #
    # NOTE: This ignores parameters if the last batch isn't complete.
    #
    NUMBER_BATCHES  = input_parameters.shape[0] // BATCH_SIZE
    
    # Track each mini-batch's training loss for analysis.
    loss_history = []

    batch_indices = np.arange( NUMBER_BATCHES )
    
    for epoch_index in range( number_epochs ):
        model.train()

        # Reset our training loss.
        running_loss = 0.0

        # "Shuffle" our training data so each epoch sees it in a different
        # order.  Note that we generate a permutation of batch indices so
        # we don't actually rearrange the training data in memory and
        # dramatically slow things down.
        permuted_batch_indices = np.random.permutation( batch_indices )
        
        for batch_index in range( NUMBER_BATCHES ):
            start_index = permuted_batch_indices[batch_index]*BATCH_SIZE
            end_index   = start_index + BATCH_SIZE
            
            # Get the next batch of droplets.
            inputs  = input_parameters[start_index:end_index, :]
            outputs = output_parameters[start_index:end_index, :]
            times   = integration_times[start_index:end_index]
            
            # Normalize the inputs and outputs to [-1, 1].
            normalized_inputs  = normalize_droplet_parameters( inputs )
            normalized_outputs = normalize_droplet_parameters( outputs )

            # XXX: Need to rethink how we handle time as an input.  This is annoying
            #      to have to stack a reshaped vector each time.
            normalized_inputs = np.hstack( (normalized_inputs,
                                            times.reshape( (BATCH_SIZE, 1) )) )
            
            normalized_inputs  = torch.from_numpy( normalized_inputs ).to( device )
            normalized_outputs = torch.from_numpy( normalized_outputs ).to( device )
                        
            # Reset the parameter gradients.
            optimizer.zero_grad()

            # Perform the forward pass.
            normalized_approximations = model( normalized_inputs )
            
            # Estimate the loss.
            loss = criterion( normalized_approximations, normalized_outputs )

            # Backwards pass and optimization.
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

            if batch_index > 0 and batch_index % MINI_BATCH_SIZE == 0:
                running_loss /= MINI_BATCH_SIZE

                print( "Epoch #{:d}, batch #{:d} loss: {:g}".format(
                    epoch_index + 1,
                    batch_index + 1,
                    running_loss ) )
                loss_history.append( running_loss )

                # Break out of the batch loop if we ever encounter an input
                # that causes the loss to spike.  Training data generation
                # should produce good inputs and outputs though should something
                # slip through we want to immediately stop training so we can
                # understand what went wrong - there is no way to recover
                # from a loss spike that is O(10) when our target loss is O(1e-4).
                if running_loss > 10:
                    print( "Crazy loss!" )
                    print( inputs, outputs )
                    break
                
                running_loss = 0.0
        else:
            # We finished all of the batches.  Adjust the learning rate and
            # go to the next epoch.
            for parameter_group in optimizer.param_groups:
                parameter_group["lr"] /= 2
            
            continue

        # We didn't complete all of the batches in this epoch.
        break
        
    # Handle the case where we didn't have enough data to complete a mini-batch.
    if len( loss_history ) == 0:
        running_loss /= number_epochs * NUMBER_BATCHES
        loss_history.append( running_loss )
        
        print( "Epoch #{:d}, batch #{:d} loss: {:g}".format(
            number_epochs,
            NUMBER_BATCHES,
            running_loss ) )
                
    return loss_history

In [ ]:
if train_model_flag:
    if training_data_path is None:
        raise ValueError( "Set training_data_path to where the training data are located!" )

    loss_history = train_model( model, 
                                criterion,
                                optimizer,
                                device,
                                number_epochs, 
                                training_data_path )

    # Plot the training loss to qualitatively assess how the model is doing.
    # Loss should consistently decrease over mini-batches.
    #
    # NOTE: This is only one aspect to evaluate a model's performance as the
    #       loss reported is on *training* data and not independent test
    #       data.  As a result, this curve will be overly optimistic.
    #
    fig_h, ax_h = plt.subplots( 1, 1, figsize=(4, 4) )

    ax_h.plot( np.log10( loss_history ), "." )
    ax_h.set_xlabel( "Mini Batch Number" )
    ax_h.set_ylabel( "log10( MSE )" )
    ax_h.set_title( "Training Loss" )

    plt.show()

    print( "Trained on {:d} mini-batches.".format(
        len( loss_history ) ) ) 

In [ ]:
if model_save_path is None:
    warnings.warn( "Please set model_path if you wish to save the model to disk.  This is intentionally *NOT* set to avoid accidentally overwriting an existing model." )
else:
    torch.save( model.state_dict(), model_save_path )
    
    print( "Saved the current model to '{:s}'.".format( model_save_path ) )

# Performance Analysis
Load a previously trained model, if requested, and qualitatively assess its performance relative
to the ODEs it was trained to approximate.

In [ ]:
if load_model_flag:
    model = SimpleNet()
    model.load_state_dict( torch.load( model_load_path ) )
    model = model.to( device )
    
    print( "Loaded a model from '{:s}'.".format( model_load_path ) )
else:
    print( "Using the previously trained model." )

In [ ]:
def do_inference( input_parameters, times, model, device ):
    """
    Estimates droplet parameters using a specified model.  Model evaluation is
    performed on the CPU
    
    Takes 4 arguments:
    
      input_parameters - NumPy array, sized number_droplets x 6, containing the
                         input parameters for number_droplets-many droplets.  These are
                         provided in their natural, physical ranges.
      times            - NumPy array, sized number_droplets, containing the integration
                         times to evaluate each droplet at.
      model            - PyTorch model to use.
      device           - XXX
    
    Returns 1 value:
    
      output_parameters - NumPy array, sized number_droplets x 2, containing the
                          estimated radius and temperature for each of the droplets
                          at the specified integraition times.  These are in their
                          natural physical ranges.

    """

    eval_model = model.to( device )
    eval_model.eval()
    
    normalized_inputs = np.hstack( (normalize_droplet_parameters( input_parameters ),
                                    times.reshape( (-1, 1) )) ).astype( "float32" )
    normalized_outputs = eval_model( torch.from_numpy( normalized_inputs ).to( device ) ).to( "cpu" ).detach().numpy()
    
    return scale_droplet_parameters( normalized_outputs )

# Verify that the model matches the ODEs outputs for a single parameter.
test_inputs      = np.array( [9.58937380e-05, 3.03864258e+02, 7.20879248e-16, 2.77617767e+02, 1.04355168e+00, 8.33695471e-01] ).reshape( (1, -1) )
test_time        = np.array( [1.0] )

t_span   = (0, 10)
solution = solve_ivp_float32_outputs( dydt,
                                      t_span,
                                      [test_inputs[0, 0], test_inputs[0, 1]],
                                      method="Radau",
                                      t_eval=test_time,
                                      args=(test_inputs[0, 2:],) )

#
# NOTE: We must transpose our expected outputs as solve_ivp() returns vectors
#       in a different direction as do_inference().
#
expected_outputs = np.array( [solution.y[0], solution.y[1]] ).T

# What does our model estimate for this set of inputs?
test_outputs = do_inference( test_inputs, test_time, model, device )

# Sanity check that we're within half a percent relative difference for radius
# and temperature.
#
# NOTE: Changing the model may require an updated tolerance as new models
#       may not have learned to approximate this particular input.  Be
#       careful when adjusting this tolerance!
#
# NOTE: We report a warning instead of raising an exception since training
#       an underperforming model shouldn't preclude the analysis and weights
#       export process.
#
if not np.allclose( test_outputs, expected_outputs, rtol=5e-3 ):
    warnings.warn( "The model did not compute the expected size/temperature ({}) for {} ({})".format(
        expected_outputs,
        test_inputs,
        test_outputs ) )

In [ ]:
def analyze_model_performance( model, input_parameters=None, figure_size=None ):
    """
    Creates a figure with four plots to qualitatively assess the supplied model's
    performance on a single droplet's parameters.  For both the radius and
    temperature variables the ODE outputs are plotted against the model's estimates.
    Additionally, the relative and absolute differences of each variable are
    plotted so fine-grained differences in the solutions can be reviewed.
    
    XXX: This should return the normalized RMSE for radius and temperature.
    
    Takes 3 arguments:
    
      model            - PyTorch model to compare against the ODEs.
      input_parameters - Optional NumPy array of input parameters to evaluate performance
                         on.  If omitted, a random set of input parameters are sampled.
      figure_size      - Optional sequence, of length 2, containing the width
                         and height of the figure created.  If omitted, defaults
                         to something big enough to comfortably assess a single
                         model's outputs.
    
    Returns nothing.
    
    """

    # Default to something that fits a set of four plots (in a 2x2 grid) comfortably.
    if figure_size is None:
        figure_size = (9, 8)
    
    # Smoothly evaluate the solution for 10 seconds into the future.
    FINAL_TIME         = 10.0
    NUMBER_TIME_POINTS = 1000

    # Specify the colors/styles in our plots.
    TRUTH_COLOR    = "b"
    MODEL_COLOR    = "r."
    RELATIVE_COLOR = "darkmagenta"
    ABSOLUTE_COLOR = "dodgerblue"
    
    # Sample the times in log-space so we get good coverage in both the DNS and
    # LES regions.
    t_eval = np.logspace( -3, np.log10( FINAL_TIME ), NUMBER_TIME_POINTS )
    
    # Get a random droplet if we weren't provided one.
    if input_parameters is None:
        # XXX: This is wasteful.  Just call np.random.uniform()
        (input_parameters,
         _,
         _,
         _,
         _) = create_droplet_batch( 1 )

    # Report the droplet we're evaluating in case we randomly sampled it.
    print( "Inputs: {}".format( input_parameters ) )
        
    # Get truth from the ODEs.
    y0           = (input_parameters[0, 0], input_parameters[0, 1])    
    solution     = solve_ivp( dydt, [0, FINAL_TIME], y0, method="Radau", t_eval=t_eval, args=(input_parameters[0, 2:],) )
    truth_output = np.vstack( (solution.y[0][:], solution.y[1][:]) ).T

    # Get the model's estimate.
    model_output = do_inference( np.tile( input_parameters, (t_eval.shape[0], 1) ),
                                 t_eval,
                                 model,
                                 "cpu" )

    # Compute the normalized RMSE across the entire time scale.
    rmse_0 = np.sqrt( np.mean( (truth_output[:, 0] - model_output[:, 0])**2 ) ) / np.mean( truth_output[:, 0] ) 
    rmse_1 = np.sqrt( np.mean( (truth_output[:, 1] - model_output[:, 1])**2 ) ) / np.mean( truth_output[:, 1] ) 
    print( "NRMSE: {:g}%, {:g}%".format( rmse_0 * 100, rmse_1 * 100) )
    
    # Compute the normalized RMSE for different regions (DNS and LES) for
    # finer-grained performance assessment.
    t_eval_mask = np.empty( (NUMBER_TIME_POINTS, 4),
                            dtype=np.bool_ )
    t_eval_mask[:, 0] = (t_eval  < 1e-2)
    t_eval_mask[:, 1] = (t_eval >= 1e-2) & (t_eval < 1e-1)
    t_eval_mask[:, 2] = (t_eval >= 1e-1) & (t_eval < 1e0)
    t_eval_mask[:, 3] = (t_eval >= 1e0) 
    
    rmse_0 = np.empty( (4,), dtype=np.float32 )
    rmse_1 = np.empty( (4,), dtype=np.float32 )
    
    for scale_index in np.arange( t_eval_mask.shape[1] ):
        rmse_0[scale_index] = np.sqrt( np.mean( (truth_output[t_eval_mask[:, scale_index], 0] - model_output[t_eval_mask[:, scale_index], 0])**2 ) ) / np.mean( truth_output[t_eval_mask[:, scale_index], 0] ) 
        rmse_1[scale_index] = np.sqrt( np.mean( (truth_output[t_eval_mask[:, scale_index], 1] - model_output[t_eval_mask[:, scale_index], 1])**2 ) ) / np.mean( truth_output[t_eval_mask[:, scale_index], 1] ) 
    
    print( "NRMSE: {}%, {}%".format( rmse_0 * 100, rmse_1 * 100 ) )
    
    # Create our figure and embed the parameters that were evaluated.
    fig_h, ax_h = plt.subplots( 2, 2, figsize=figure_size )
    fig_h.suptitle( "Droplet Size and Temperature\nRadius={:g}, Temperature={:g}, m_s={:g}, Air Temp={:g}, RH={:g}, rhoa={:g}".format(
        input_parameters[0, 0],
        input_parameters[0, 1],
        input_parameters[0, 2],
        input_parameters[0, 3],
        input_parameters[0, 4],
        input_parameters[0, 5]
        ) )

    # Truth vs model predictions.
    ax_h[0][0].plot( t_eval, truth_output[:, 0], TRUTH_COLOR,
                     t_eval, model_output[:, 0], MODEL_COLOR )
    ax_h[0][1].plot( t_eval, truth_output[:, 1], TRUTH_COLOR,
                     t_eval, model_output[:, 1], MODEL_COLOR )

    # Relative difference between truth and model.
    ax_h[1][0].plot( t_eval, 
                     np.abs( truth_output[:, 0] - model_output[:, 0] ) / truth_output[:, 0] * 100,
                     color=RELATIVE_COLOR )
    ax_h[1][0].tick_params( axis="y", labelcolor=RELATIVE_COLOR )
    ax_h[1][1].plot( t_eval, 
                     np.abs( truth_output[:, 1] - model_output[:, 1] ) / truth_output[:, 1] * 100,
                     color=RELATIVE_COLOR )
    ax_h[1][1].tick_params( axis="y", labelcolor=RELATIVE_COLOR )

    # Aboslute difference between truth and model.
    ax_h_twin_radius = ax_h[1][0].twinx()
    ax_h_twin_radius.plot( t_eval, np.abs( truth_output[:, 0] - model_output[:, 0] ), color=ABSOLUTE_COLOR )
    ax_h_twin_radius.set_ylabel( "Absolute Difference (m)" )
    ax_h_twin_radius.tick_params( axis="y", labelcolor=ABSOLUTE_COLOR )

    ax_h_twin_temperature = ax_h[1][1].twinx()
    ax_h_twin_temperature.plot( t_eval, np.abs( truth_output[:, 1] - model_output[:, 1] ), color=ABSOLUTE_COLOR )
    ax_h_twin_temperature.set_ylabel( "Absolute Difference (K)" )
    ax_h_twin_temperature.tick_params( axis="y", labelcolor=ABSOLUTE_COLOR )

    # Label the comparison plots' lines.
    ax_h[0][0].legend( ["Truth", "Model"] )
    ax_h[0][1].legend( ["Truth", "Model"] )

    # Label the columns of plots.
    ax_h[0][0].set_title( "Radius" )
    ax_h[0][1].set_title( "Temperature" )
    
    ax_h[0][0].set_ylabel( "Radius (m)" )
    ax_h[0][1].set_ylabel( "Temperature (K)" )
    ax_h[1][0].set_ylabel( "Relative Difference (%)" )
    ax_h[1][0].set_xlabel( "Time (s)" )
    ax_h[1][1].set_ylabel( "Relative Difference (%)" )
    ax_h[1][1].set_xlabel( "Time (s)" )

    # Show time in log-scale as well as the droplets' radius.
    ax_h[0][0].set_xscale( "log" )
    ax_h[0][0].set_yscale( "log" )
    ax_h[0][1].set_xscale( "log" )
    ax_h[0][1].set_yscale( "log" )
    ax_h[1][0].set_xscale( "log" )
    ax_h[1][1].set_xscale( "log" )
    
    fig_h.tight_layout()


In [ ]:
input_parameters = scale_droplet_parameters( np.random.uniform( low=-1.0, high=1.0, size=(1, 6) ).astype( "float32" ) )
analyze_model_performance( model, input_parameters )

# Serializing Model Weights into Fortran
Generate a simple export of the model's weight into a format that is usable for a naive implementation of inference with SimpleNet.

In [ ]:
def generate_fortran_module( model_name, model_state, output_path ):
    """
    Creates a Fortran 2003 module that allows use of the supplied model with a
    batch size of 1 during inference.
    
    NOTE: This currently expects the supplied model state to represent a 4-layer
          MLP with a specific set weights/biases names.
    
    Takes 2 arguments:
    
      model_state     - PyTorch model state dictionary for the model to expose.
      output_path     - File to write to.  If this exists it is overwritten.
      
    Returns nothing.
    
    """

    #
    # NOTE: This is hard coded against the SimpleNet class' implementation.  This
    #       strange coupling could be better (less strange?) by moving this routine
    #       into SimpleNet.
    #
    expected_weights = ["fc1.weight",
                        "fc2.weight",
                        "fc3.weight",
                        "fc4.weight"]
    expected_biases  = ["fc1.bias",
                        "fc2.bias",
                        "fc3.bias",
                        "fc4.bias"]
    
    def write_module_prolog( model_name, model_state, output_fp ):
        """     
        Writes the module's prolog to the supplied file handle.

        Takes 3 arguments:
        
          model_name  - Name of the model to write in the generated module's comments
                        so as to identify where the weights came from.
          model_state - PyTorch model state dictionary for the model exposed by the module.
          output_fp   - File handle to write to.
          
        Returns nothing.

        """
           
        import datetime
            
        expected_parameters = set( expected_weights + expected_biases )
        
        # Confirm we have the weights and biases that we expect.
        if expected_parameters != set( model_state.keys() ):
            raise ValueError( "Model provided does not match a 4-layer MLP!" )
        
        # Ensure that the weights are 2D matrices.
        #
        # NOTE: We could be more thorough to ensure that the output of one layer
        #       matches the input of the next layer but we're in a rush right now...
        #
        for weights_name in expected_weights:
            if len( model_state[weights_name].shape ) != 2:
                raise ValueError( "'{:s}' is not a rank-2 tensor!".format( weights_name ) )
                
        # Ensure that the biases are vectors.
        #
        # NOTE: We could be more thorough to ensure that the output of one layer
        #       matches the input of the next layer but we're in a rush right now...
        #
        for bias_name in expected_biases:
            if len( model_state[bias_name].shape ) != 1:
                raise ValueError( "'{:s}' is not a rank-1 tensor!".format( bias_name ) )
        
        # Get the current date so people have an idea of when/how this module was
        # created.
        right_now             = datetime.datetime.now()
        current_date_time_str = right_now.strftime( "%Y/%m/%d at %H:%M:%S" )
        
        preamble_str = """!
! NOTE: This module was generated from {model_name:s} on {creation_date:s}.
!       Do not modify this directly!  Update the module generator's template and
!       regenerate this file!
!

module droplet_model

    ! Module containing a model of droplet size and temperature using a 4-layer
    ! multi-layer perceptron (MLP) neural network.  Once initialized, a single
    ! droplet's parameters and timestep can be used to estimate its future size
    ! and temperature.  This approach avoids an iterative search using
    ! Gauss-Newton to estimate its future charueacteristics and is typically much
    ! faster (O(30x) observed for small parameter count MLPs).
    !
    ! The module must be initialized with a call to initialize_model() and then
    ! droplet estimation, estimate(), can be performed as many times as
    ! necessary.  Weights for the MLP are hardcoded so initialization does not
    ! require any external resources.  An example calling sequence is the
    ! following:
    !
    !     ! Somewhere during simulation startup.
    !     call initialize_model()
    !
    !     ...
    !
    !     ! Particle simulation code.
    !     real*4 :: input_parameters(NUMBER_INPUTS)
    !     real*4 :: output_parameters(NUMBER_OUTPUTS)
    !
    !     do particle_index = 1, number_particles
    !
    !         input_parameters = [radius, temperature, salinity, air_temperature, rh, rhoa, t_final]
    !         call estimate( input_parameters, output_parameters )
    !
    !         new_radius      = output_parameters(RADIUS_INDEX)
    !         new_temperature = output_parameters(TEMPERATURE_INDEX)
    !
    !     end do
    !
    ! Inputs to the estimation, as are outputs, are provided in physical units
    ! and are internally normalized before using the model allowing this to be a
    ! drop-in replacement for alternative approaches for estimating droplet
    ! characteristics.
    !
    ! The model was trained on droplet parameters sampled from the product space
    ! of each of the individual parameters.  While this simple approach allowed
    ! for rapid development of a prototype it does include combinations of
    ! parameters that may be physically impossible which may have been
    ! under-sampled during training.  As a result the estimations in these
    ! corners of the parameter space may be of lower accuracy than more sampled,
    ! better behaved regions of the space.  The initial parameter ranges were
    ! selected by David Richter during prototyping and aim to cover all
    ! simulations of interest.
    !
    ! Things to note:
    !
    !   1. This approach was developed for code that would operate on a single
    !      droplet at a time vs multiple droplets at once.  As a result the
    !      application of a MLP network is simply a series of matrix-vector
    !      multiplications, vector-vector additions, and an activation function
    !      applied to the result.  We implement this case rather than attempt to
    !      generalize as the performance gain is already non-trivial and it is
    !      unclear when the code (if ever) would support batch estimation.
    !
    !   2. We could statically initialize each layer's weights and biases at
    !      compile time, albeit at the expense of readability.  By separating
    !      the weights and biases' definitions from their values we retain the
    !      ability to understand the structure and internal components of the
    !      MLP while requiring a single subroutine call at startup.
    !
    !   3. This is a naive implementation of a 4-layer MLP (it was written in 15
    !      minutes!)  and does not implement all potential optimizations.  In
    !      particular, each intermediate layer has its own array and no attempt
    !      to reuse arrays or work out of one large array has been made, so as
    !      to minimize the MLP's run-time footprint.  As initially developed the
    !      MLP is very small (O(2500) parameters) and uses O(10 KiB) storage,
    !      though should this ever be used for much larger MLPs, or with larger
    !      batch sizes (see above) then some of the omitted optimizations should
    !      be revisited.
    !
    !   4. While the MLP was trained on the product space of the input
    !      parameters and has poor performance on certain (likely) physically
    !      impossible parameter combinations, it is unclear what need to be done
    !      to improve the situation.  Re-training on a subset of the input
    !      parameters might improve performance and could be done without
    !      changing this implementation (albeit with undefined behavior for any
    !      physically impossible input) but it remains to be seen how much
    !      accuracy would be gained from this.
    !
    !   5. This approach is *NOT* OpenMP safe and is inherently single-threaded!
    !      Should multiple estimations need to be performed in parallel, an
    !      additional interface is needed that provides the intermediate layers
    !      to the estimation subroutine.  All of the weights and biases are
    !      effectively read-only and can be shared across threads.  This is
    !      straight forward to implement though it was not necessary for the
    !      initial implementation.
    !
    !   6. Single precision floating point is used throughout the module as it
    !      has been deemed to produce an acceptable level of accuracy for the
    !      radius and temperature parameters.  The input and output arrays
    !      must be real*4.
    !
    ! NOTE: This file was automatically generated from the the MLP's training
    !       code!  Do not modify this unless you really know what you're doing!
    !

    ! Indices into the input and output vectors.  The input vector uses all of
    ! the indices while the output vector only uses the first two.
    integer, parameter :: RADIUS_INDEX          = 1
    integer, parameter :: TEMPERATURE_INDEX     = 2
    integer, parameter :: SALINITY_INDEX        = 3
    integer, parameter :: AIR_TEMPERATURE_INDEX = 4
    integer, parameter :: RH_INDEX              = 5
    integer, parameter :: RHOA_INDEX            = 6
    integer, parameter :: TFINAL_INDEX          = 7

    integer, parameter :: NUMBER_INPUTS  = {number_inputs:d}
    integer, parameter :: NUMBER_OUTPUTS = {number_outputs:d}

    integer, parameter :: NUMBER_HIDDEN_LAYER1_NEURONS = {number_layer1_neurons:d}
    integer, parameter :: NUMBER_HIDDEN_LAYER2_NEURONS = {number_layer2_neurons:d}
    integer, parameter :: NUMBER_HIDDEN_LAYER3_NEURONS = {number_layer3_neurons:d}

    ! The input variables must be normalized into the range of [-1, 1] before
    ! feeding them into the model.  The following ranges, means, and widths are
    ! used to map into, and from, the range of [-1, 1].
    !
    ! NOTE: These *must* match the training data!  Do not change these without
    !       retraining the model!
    !
    real*4, parameter :: RADIUS_LOG_RANGE(2)      = [{radius_start:.1f}, {radius_end:.1f}]
    real*4, parameter :: TEMPERATURE_RANGE(2)     = [{temperature_start:.1f}, {temperature_end:.1f}]
    real*4, parameter :: SALINITY_LOG_RANGE(2)    = [{salinity_start:.1f}, {salinity_end:.1f}]
    real*4, parameter :: AIR_TEMPERATURE_RANGE(2) = [{air_temperature_start:.1f}, {air_temperature_end:.1f}]
    real*4, parameter :: RH_RANGE(2)              = [{rh_start:.2f}, {rh_end:.2f}]
    real*4, parameter :: RHOA_RANGE(2)            = [{rhoa_start:.1f}, {rhoa_end:.1f}]

    real*4, parameter :: RADIUS_LOG_MEAN      = SUM( RADIUS_LOG_RANGE ) / 2
    real*4, parameter :: TEMPERATURE_MEAN     = SUM( TEMPERATURE_RANGE ) / 2
    real*4, parameter :: SALINITY_LOG_MEAN    = SUM( SALINITY_LOG_RANGE ) / 2
    real*4, parameter :: AIR_TEMPERATURE_MEAN = SUM( AIR_TEMPERATURE_RANGE ) / 2
    real*4, parameter :: RH_MEAN              = SUM( RH_RANGE ) / 2
    real*4, parameter :: RHOA_MEAN            = SUM( RHOA_RANGE ) / 2

    !
    ! NOTE: We have be careful about lines no longer than 132 characters so we
    !       eliminate whitespace in the expressions.
    !
    real*4, parameter :: RADIUS_LOG_WIDTH      = (RADIUS_LOG_RANGE(2)-RADIUS_LOG_RANGE(1))/2
    real*4, parameter :: TEMPERATURE_WIDTH     = (TEMPERATURE_RANGE(2)-TEMPERATURE_RANGE(1))/2
    real*4, parameter :: SALINITY_LOG_WIDTH    = (SALINITY_LOG_RANGE(2)-SALINITY_LOG_RANGE(1))/2
    real*4, parameter :: AIR_TEMPERATURE_WIDTH = (AIR_TEMPERATURE_RANGE(2)-AIR_TEMPERATURE_RANGE(1))/2
    real*4, parameter :: RH_WIDTH              = (RH_RANGE(2)-RH_RANGE(1))/2
    real*4, parameter :: RHOA_WIDTH            = (RHOA_RANGE(2)-RHOA_RANGE(1))/2

    ! Weights for each of the layers.
    real*4, dimension(NUMBER_INPUTS, NUMBER_HIDDEN_LAYER1_NEURONS)                :: layer1_weights
    real*4, dimension(NUMBER_HIDDEN_LAYER1_NEURONS, NUMBER_HIDDEN_LAYER2_NEURONS) :: layer2_weights
    real*4, dimension(NUMBER_HIDDEN_LAYER2_NEURONS, NUMBER_HIDDEN_LAYER3_NEURONS) :: layer3_weights
    real*4, dimension(NUMBER_HIDDEN_LAYER3_NEURONS, NUMBER_OUTPUTS)               :: output_weights

    ! Biases for each of the layers.
    real*4, dimension(NUMBER_HIDDEN_LAYER1_NEURONS) :: layer1_biases
    real*4, dimension(NUMBER_HIDDEN_LAYER2_NEURONS) :: layer2_biases
    real*4, dimension(NUMBER_HIDDEN_LAYER3_NEURONS) :: layer3_biases
    real*4, dimension(NUMBER_OUTPUTS)               :: output_biases

    ! Arrays to hold the intermediate results between layers.
    !
    ! NOTE: We don't do anything fancy like detecting when we could reuse an
    !       intermediate.
    !
    real*4, dimension(NUMBER_HIDDEN_LAYER1_NEURONS) :: layer1_intermediate
    real*4, dimension(NUMBER_HIDDEN_LAYER2_NEURONS) :: layer2_intermediate
    real*4, dimension(NUMBER_HIDDEN_LAYER3_NEURONS) :: layer3_intermediate

    contains
""".format(
    # XXX: Double check the hidden layer neuron's sizes!  This was not
    #      thoroughly tested since the target model had identical sizes
    #      throughout.
    model_name=model_name,
    creation_date=current_date_time_str,
    number_inputs=model_state["fc1.weight"].shape[1],
    number_outputs=model_state["fc4.weight"].shape[0],
    number_layer1_neurons=model_state["fc1.weight"].shape[0],
    number_layer2_neurons=model_state["fc2.weight"].shape[0],
    number_layer3_neurons=model_state["fc3.weight"].shape[0],
    radius_start=DROPLET_RADIUS_LOG_RANGE[0],
    radius_end=DROPLET_RADIUS_LOG_RANGE[1],
    temperature_start=DROPLET_TEMPERATURE_RANGE[0],
    temperature_end=DROPLET_TEMPERATURE_RANGE[1],
    salinity_start=DROPLET_SALINITY_LOG_RANGE[0],
    salinity_end=DROPLET_SALINITY_LOG_RANGE[1],
    air_temperature_start=DROPLET_AIR_TEMPERATURE_RANGE[0],
    air_temperature_end=DROPLET_AIR_TEMPERATURE_RANGE[1],
    rh_start=DROPLET_RELATIVE_HUMIDITY_RANGE[0],
    rh_end=DROPLET_RELATIVE_HUMIDITY_RANGE[1],
    rhoa_start=DROPLET_RHOA_RANGE[0],
    rhoa_end=DROPLET_RHOA_RANGE[1]
        )
        
        print( preamble_str, file=output_fp )

    def write_model_initializaiton_routine( model_state, output_fp, indentation_str ):
        """
        Writes the model's initialization routine to the supplied file handle with
        a specific amount of indentation.
        
        Takes 3 arguments:
        
          model_state     - PyTorch model state dictionary for the model to initialize.
          output_fp       - File handle to write to.
          indentation_str - String prepended to each line of the subroutine written.
          
        Returns nothing.

        """

        def write_model_weights( model_state, output_fp, indentation_str ):
            """
            Internal routine that generates the weight initialization expressions for
            all of the weights in the supplied model state.  This does not generate
            initialization for biases nor does it generate a fully functional subroutine
            (i.e. pre-amble/epilog).
            
            Takes 3 arguments:
            
              model_state     - PyTorch model state dictionary for the model to initialize.
              output_fp       - File handle to write to.
              indentation_str - String prepended to each line of the subroutine written.

            Returns nothing.
            """
            
            # We rename each of the weights to Fortran-compatible symbols that are
            # self-descriptive.
            original_layer_names = expected_weights
            new_layer_names      = ["layer1_weights",
                                    "layer2_weights",
                                    "layer3_weights",
                                    "output_weights"]
            
            for original_layer_name, new_layer_name in zip( original_layer_names, new_layer_names ):
                layer_parameters = model_state[original_layer_name]
                
                # Get the shape of the array while reversing the order from 
                # row-major (C, Python) to column-major (Fortran)
                outer_size, inner_size = layer_parameters.shape
                
                # Start the array assignment to this weight's variable.
                weights_definition_str = "{:s}{:s} = reshape( [".format(
                    indentation_str,
                    new_layer_name )
                
                # Create enough indentation so all of the weight's values are aligned
                # at the first column after the open bracket.
                weights_indentation_str = " " * len( weights_definition_str ) 
                
                # Template to join successive weight's values together with.  Note that
                # this is not applied to the last one.
                definition_join_str    = ", &\n{:s}".format( weights_indentation_str )
                
                # Build the list of indented weight values, each printed with 10 digits
                # after the decimal point, and aligned so that positive weights have a
                # leading space to match negative weights' alignment.  We ignore the
                # weight's shape since we'll reshape a 1D vector at run-time of the
                # compiled Fortran executable.
                #
                # NOTE: This assumes the weights values' magnitudes are such that 10 digits
                #       of precision is sufficient.
                #
                weights_values_str = definition_join_str.join( 
                    map( lambda x: "{: .10f}".format( x ),
                         layer_parameters.cpu().numpy().ravel() ) )

                # Create the dimensions array supplied to reshape(), as well as the closing
                # parenthsis.
                weights_reshape_dimensions_str = "[{:d}, {:d}] )".format(
                    inner_size,
                    outer_size )

                print( "{:s}{:s}], &\n{:s}{:s}".format(
                    weights_definition_str,
                    weights_values_str,
                    weights_indentation_str,
                    weights_reshape_dimensions_str ),
                      file=output_fp )

        def write_model_biases( model_state, output_fp, indentation_str ):
            """
            Internal routine that generates the biases initialization expressions for
            all of the biases in the supplied model state.  This does not generate
            initialization for weights nor does it generate a fully functional subroutine
            (i.e. pre-amble/epilog).
            
            Takes 3 arguments:
            
              model_state     - PyTorch model state dictionary for the model to initialize.
              output_fp       - File handle to write to.
              indentation_str - String prepended to each line of the subroutine written.

            Returns nothing.
            """
            
            # We rename each of the biases to Fortran-compatible symbols that are
            # self-descriptive.
            original_layer_names = expected_biases
            new_layer_names      = ["layer1_biases",
                                    "layer2_biases",
                                    "layer3_biases",
                                    "output_biases"]
            
            for original_layer_name, new_layer_name in zip( original_layer_names, new_layer_names ):
                layer_parameters = model_state[original_layer_name]
                
                # Start the array assignment to this bias' variable.
                biases_definition_str = "{:s}{:s} = [".format(
                    indentation_str,
                    new_layer_name )
                
                # Create enough indentation so all of the bias values are aligned
                # at the first column after the open bracket.
                biases_indentation_str = " " * len( biases_definition_str ) 
                
                # Template to join successive bias values together with.  Note that
                # this is not applied to the last one.
                definition_join_str    = ", &\n{:s}".format( biases_indentation_str )
                
                # Build the list of indented bias values, each printed with 10 digits
                # after the decimal point, and aligned so that positive biases have a
                # leading space to match negative biases' alignment.
                #
                # NOTE: This assumes the bias values' magnitudes are such that 10 digits
                #       of precision is sufficient.
                #
                bias_values_str = definition_join_str.join( 
                    map( lambda x: "{: .10f}".format( x ),
                         layer_parameters.cpu().numpy() ) )
                
                print( "{:s}{:s}]".format(
                    biases_definition_str,
                    bias_values_str,
                    biases_indentation_str ),
                      file=output_fp )
        
        # Our initialization routine is inside of a "contains" block so we're
        # indented one level deeper than the caller.
        inner_indentation_str = "    {:s}".format( indentation_str )
        
        # Prolog and epilog of our initialization routine, along with extra
        # newlines so we survive the application of .splitlines() below.
        initialization_prolog_str = "subroutine initialize_model()\n\n    implicit none\n\n"
        initialization_epilog_str = "\nend subroutine initialize_model"
        
        # Write the subroutine's prolog.
        for initialization_line in initialization_prolog_str.splitlines():
            print( "{:s}{:s}".format(
                indentation_str if len( initialization_line ) > 0 else "",
                initialization_line ),
                file=output_fp )
            
        # Write the weights and biases separated by an empty line.
        write_model_weights( model_state, output_fp, inner_indentation_str )
        print( "", file=output_fp )
        write_model_biases( model_state, output_fp, inner_indentation_str )
        
        # Write the subroutine's epilog.
        for initialization_line in initialization_epilog_str.splitlines():
            print( "{:s}{:s}".format(
                indentation_str if len( initialization_line ) > 0 else "",
                initialization_line ),
                file=output_fp )

    
    def write_model_inference_routine( model_state, output_fp, indentation_str ):
        """
        Writes the model's inference routine to the supplied file handle with
        a specific amount of indentation.
        
        Takes 3 arguments:
        
          model_state     - Unused argument kept for a consistent signature.
          output_fp       - File handle to write to.
          indentation_str - String prepended to each line of the subroutine written.

        Returns nothing.

        """
        
        inference_str = """
! Estimates a droplet's future size and temperature based on it's current size
! and temperature, as well as key parameters from the environment its in.
! Inputs and outputs are provided/returned in un-normalized physical scales.
subroutine estimate( input, output )

    implicit none

    !
    ! NOTE: This is hardcoded to assume a batch size of 1 so we can have
    !       a 1D array vs dealing with a 2D array with a singleton
    !       dimension.
    !
    real*4, dimension(NUMBER_INPUTS), intent(in)   :: input
    real*4, dimension(NUMBER_OUTPUTS), intent(out) :: output

    real*4, dimension(NUMBER_INPUTS)               :: normalized_input
    integer                                        :: output_index

    ! Normalize the non-temporal inputs so they're in the range [-1, 1].
    normalized_input(RADIUS_INDEX)          = (log10( input(RADIUS_INDEX) ) - RADIUS_LOG_MEAN) / RADIUS_LOG_WIDTH
    normalized_input(TEMPERATURE_INDEX)     = (input(TEMPERATURE_INDEX) - TEMPERATURE_MEAN) / TEMPERATURE_WIDTH
    normalized_input(SALINITY_INDEX)        = (log10( input(SALINITY_INDEX) ) - SALINITY_LOG_MEAN) / SALINITY_LOG_WIDTH
    normalized_input(AIR_TEMPERATURE_INDEX) = (input(AIR_TEMPERATURE_INDEX) - AIR_TEMPERATURE_MEAN) / AIR_TEMPERATURE_WIDTH
    normalized_input(RH_INDEX)              = (input(RH_INDEX) - RH_MEAN) / RH_WIDTH
    normalized_input(RHOA_INDEX)            = (input(RHOA_INDEX) - RHOA_MEAN) / RHOA_WIDTH

    ! Our integration time remains as is.
    normalized_input(TFINAL_INDEX)          = input(TFINAL_INDEX)

    ! Compute x_1 = ReLU( W_1*I + b_1 ).
    do output_index = 1, NUMBER_HIDDEN_LAYER1_NEURONS
        layer1_intermediate(output_index) = &
             max( sum( normalized_input(:) * layer1_weights(:, output_index) ) + layer1_biases(output_index), 0.0 )
    end do

    ! Compute x_2 = ReLU( W_2*x_1 + b_2 ).
    do output_index = 1, NUMBER_HIDDEN_LAYER2_NEURONS
        layer2_intermediate(output_index) = &
             max( sum( layer1_intermediate(:) * layer2_weights(:, output_index) ) + layer2_biases(output_index), 0.0 )
    end do

    ! Compute x_3 = ReLU( W_3*x_2 + b_3 ).
    do output_index = 1, NUMBER_HIDDEN_LAYER3_NEURONS
        layer3_intermediate(output_index) = &
             max( sum( layer2_intermediate(:) * layer3_weights(:, output_index) ) + layer3_biases(output_index), 0.0 )
    end do

    ! Compute O = W_4*x_3 + b_4.
    do output_index = 1, NUMBER_OUTPUTS
        output(output_index) = sum( layer3_intermediate(:) * output_weights(:, output_index) ) + output_biases(output_index)
    end do

    ! Scale the outputs to the expected ranges.
    output(RADIUS_INDEX)      = 10.0**(output(RADIUS_INDEX) * RADIUS_LOG_WIDTH + RADIUS_LOG_MEAN)
    output(TEMPERATURE_INDEX) = output(TEMPERATURE_INDEX) * TEMPERATURE_WIDTH + TEMPERATURE_MEAN

end subroutine estimate

"""

        for inference_line in inference_str.splitlines():
            print( "{:s}{:s}".format( 
                indentation_str if len( inference_line ) > 0 else "",
                inference_line ),
                   file=output_fp )
    
    def write_module_epilog( model_state, output_fp ):
        """
        Writes the module's epilog to the supplied file handle.
        
        Takes 2 arguments:
        
          model_state     - Unused argument kept for a consistent signature.
          output_fp       - File handle to write to.
          
        Returns nothing.

        """

        epilog_str = "end module droplet_model"

        print( epilog_str, file=output_fp )
    
    # Module subroutines, inside of a "contains" block, are indented twice.
    indentation_str = " " * 8
    
    with open( output_path, "w" ) as output_fp:
        # Write out the beginning of the module.  This includes the definitions for
        # the layers' weights and biases but not the method that initializes them.
        write_module_prolog( model_name, model_state, output_fp )

        # Write out both of the subroutines that comprise this model.
        write_model_initializaiton_routine( model_state, output_fp, indentation_str )
        write_model_inference_routine( model_state, output_fp, indentation_str )
        
        # Write out the end of the module.
        write_module_epilog( model_state, output_fp )

In [ ]:
# Overwrite the module used by NTLP so future simulations use the current weights.
if write_weights_flag:
    
    # Pick the most descriptive name we have so researchers have an idea where
    # the weights came from.
    if load_model_flag:
        exported_model_name = model_load_path.split( "/" )[-1]
    else:
        exported_model_name = model_name
    
    exported_file_path = "../../droplet_model.f90"
    
    generate_fortran_module( exported_model_name,
                             model.state_dict(),
                             exported_file_path )
    
    print( "Wrote model weights and inferencing code to '{:s}'.".format(
        exported_file_path ) )
else:
    print( "Skipped writing model weights." )